In [2]:
import pandas as pd
import requests
import unicodedata
import time

class MexicanDroughtIndex:
    def __init__(self, msm_path="MunicipiosSequia.xlsx"):
        """
        Initializes the index by loading the official CONAGUA drought table.
        """
        print("Loading Drought Data...")
        # Load the Excel file (assuming 'MUNICIPIOS' is the sheet name)
        # If using the CSV version, change to pd.read_csv()
        try:
            self.df = pd.read_excel(msm_path, sheet_name="MUNICIPIOS")
        except:
            # Fallback if user converted to CSV
            self.df = pd.read_csv("MunicipiosSequia.xlsx - MUNICIPIOS.csv")

        # --- 1. Dynamic Date Extraction ---
        # Find all columns that look like dates (YYYY-MM-DD)
        # We assume they start with "20"
        date_cols = [c for c in self.df.columns if str(c).strip().startswith('20')]

        # Sort and pick the LAST TWO available periods
        self.target_dates = sorted(date_cols)[-2:]
        print(f"Tracking Drought for periods: {self.target_dates}")

        # --- 2. Pre-process Names for Matching ---
        # Create a clean column for matching (UPPERCASE, NO ACCENTS)
        self.df['MATCH_KEY'] = self.df['NOMBRE_MUN'].apply(self._clean_text)
        self.df['ENTIDAD_KEY'] = self.df['ENTIDAD'].apply(self._clean_text)

        # Mapping D0-D4 to numeric risk (0-5)
        self.risk_map = {
            'D0': 1, 'D1': 2, 'D2': 3, 'D3': 4, 'D4': 5,
            'NaN': 0, 'Sin Sequia': 0
        }

    def _clean_text(self, text):
        """Standardizes text: 'Asientos' -> 'ASIENTOS', 'Nuñez' -> 'NUNEZ'"""
        if pd.isna(text): return ""
        text = str(text)
        # Remove accents
        text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
        return text.upper().strip()

    def get_location_name(self, lat, lon):
        """
        Uses OpenStreetMap API to get Municipality and State from Lat/Lon.
        Rate Limit: 1 request per second.
        """
        url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}"
        headers = {'User-Agent': 'MexicoDroughtApp/1.0'} # Required by OSM

        try:
            response = requests.get(url, headers=headers)
            data = response.json()

            address = data.get('address', {})
            # OSM keys for municipality can vary: 'county', 'city', 'town', 'village'
            municipality = address.get('county', address.get('city', address.get('town', '')))
            state = address.get('state', '')

            return municipality, state
        except Exception as e:
            print(f"API Error: {e}")
            return None, None

    def get_risk_vector(self, lat, lon):
        """
        Main function to get the vector for your ML model.
        Returns: [Drought_Last_Month, Drought_This_Month]
        """
        # 1. Get Location Name from API
        mun_name, state_name = self.get_location_name(lat, lon)

        if not mun_name:
            return [0, 0] # API failed or location not found

        # 2. Clean Names
        clean_mun = self._clean_text(mun_name)
        # Remove common prefixes from API (e.g., "Municipio de Monterrey" -> "Monterrey")
        clean_mun = clean_mun.replace("MUNICIPIO DE ", "").replace("MUNICIPIO ", "")

        # 3. Lookup in DataFrame
        # Filter by Municipality Name
        match = self.df[self.df['MATCH_KEY'] == clean_mun]

        # Optional: Filter by State if multiple municipalities have same name
        # clean_state = self._clean_text(state_name)
        # match = match[match['ENTIDAD_KEY'] == clean_state]

        if match.empty:
            print(f"Warning: Could not match '{clean_mun}' in database.")
            return [0, 0]

        # 4. Extract Values
        row = match.iloc[0]
        val1 = row[self.target_dates[0]] # 2 months ago
        val2 = row[self.target_dates[1]] # Last month

        # Convert to numeric
        score1 = self.risk_map.get(str(val1), 0)
        score2 = self.risk_map.get(str(val2), 0)

        return [score1, score2]

# --- USAGE EXAMPLE ---
if __name__ == "__main__":
    # Initialize Engine
    engine = MexicanDroughtIndex("MunicipiosSequia.xlsx")

    # Test: Monterrey, NL (Lat: 25.6866, Lon: -100.3161)
    # The API will find "Monterrey", the code will look it up in the Excel.
    user_lat = 25.6866
    user_lon = -100.3161

    risk_vector = engine.get_risk_vector(user_lat, user_lon)

    print(f"\nPrediction Vector for ({user_lat}, {user_lon}):")
    print(f"  > Date {engine.target_dates[0]}: Level {risk_vector[0]}")
    print(f"  > Date {engine.target_dates[1]}: Level {risk_vector[1]}")
    print(f"  > Model Input: {risk_vector}")


Loading Drought Data...
Tracking Drought for periods: [datetime.datetime(2025, 12, 31, 0, 0), datetime.datetime(2026, 1, 15, 0, 0)]

Prediction Vector for (25.6866, -100.3161):
  > Date 2025-12-31 00:00:00: Level 0
  > Date 2026-01-15 00:00:00: Level 0
  > Model Input: [0, 0]


In [3]:
import pandas as pd
import numpy as np
import unicodedata

# --- CONFIGURATION ---
MSM_FILE = "MunicipiosSequia.xlsx - MUNICIPIOS.csv"
TARGET_DATES = ['2025-12-31', '2026-01-15']
CDMX_ALCALDIAS = [
    'Azcapotzalco', 'Coyoacán', 'Cuajimalpa de Morelos', 'Gustavo A. Madero',
    'Iztacalco', 'Iztapalapa', 'La Magdalena Contreras', 'Milpa Alta',
    'Álvaro Obregón', 'Tláhuac', 'Tlalpan', 'Xochimilco', 'Benito Juárez',
    'Cuauhtémoc', 'Miguel Hidalgo', 'Venustiano Carranza'
]

# --- 1. HELPER FUNCTIONS ---

def clean_text(text):
    """Normalize text for matching (e.g., 'Coyoacán' -> 'COYOACAN')"""
    if pd.isna(text): return ""
    text = str(text)
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return text.upper().strip()

def load_msm_data(filepath, dates):
    """Extracts drought levels specifically for CDMX from the file."""
    try:
        df = pd.read_csv(filepath)
        # Normalize names for matching
        df['MATCH_KEY'] = df['NOMBRE_MUN'].apply(clean_text)

        # Dictionary to store {Municipality: {Date: Level}}
        msm_lookup = {}

        for mun in CDMX_ALCALDIAS:
            clean_mun = clean_text(mun)
            match = df[df['MATCH_KEY'] == clean_mun]

            if not match.empty:
                # Map D0-D4 to 1-5, NaNs to 0
                risk_map = {'D0': 1, 'D1': 2, 'D2': 3, 'D3': 4, 'D4': 5}

                levels = {}
                for d in dates:
                    val = match.iloc[0].get(d, 'NaN')
                    levels[d] = risk_map.get(str(val), 0) # Default to 0 if NaN
                msm_lookup[mun] = levels
            else:
                msm_lookup[mun] = {d: 0 for d in dates} # Default if not found

        return msm_lookup
    except Exception as e:
        print(f"Warning: Could not load MSM file ({e}). Using Mock Data.")
        # Fallback Mock Data
        return {mun: {d: np.random.randint(0, 3) for d in dates} for mun in CDMX_ALCALDIAS}

# --- 2. DATASET GENERATION ---

def create_training_dataset():
    print(f"Generating Training Data for {len(CDMX_ALCALDIAS)} CDMX Zones...")

    # Load real static risks
    msm_data = load_msm_data(MSM_FILE, TARGET_DATES)

    dataset_rows = []

    for date in TARGET_DATES:
        for mun in CDMX_ALCALDIAS:

            # --- A. STATIC / SEMI-STATIC INDEX VARIABLES ---
            drought_level = msm_data[mun][date]

            # WRI Score: CDMX is generally High Stress (3.0 - 5.0)
            # In production, use the Shapefile lookup. Here we simulate variance per zone.
            # Iztapalapa/Gustavo A Madero (East) usually have higher stress than West.
            base_wri = 4.0 if mun in ['Iztapalapa', 'Tláhuac', 'Iztacalco'] else 3.0
            wri_score = np.clip(np.random.normal(base_wri, 0.5), 0, 5)

            # --- B. DYNAMIC VARIABLES (The "Sensors") ---
            # We simulate these based on your provided JSON to create realistic training variance

            # 1. Weather (Hard Sensors)
            # Simulating: '2025-12-31' (Winter/Dry) vs Rain Events
            is_dry_season = 1 if '-12-' in date or '-01-' in date else 0

            precip_7d = np.random.exponential(5) if is_dry_season else np.random.exponential(45)
            days_no_rain = np.random.randint(0, 60) if is_dry_season else np.random.randint(0, 5)

            # 2. Social (Soft Sensors) - The "Crowdfunding" Signal
            # Higher population zones (Iztapalapa) get more reports
            pop_factor = 2.0 if mun == 'Iztapalapa' else 1.0
            report_count = int(np.random.poisson(5 * pop_factor))

            # Logic: If drought is high, reports go up
            if drought_level >= 3:
                report_count += np.random.randint(10, 50)

            # --- C. CONSTRUCT THE ROW ---
            row = {
                # Identifiers
                'location_name': mun,
                'date': date,

                # --- BLOCK 1: HARD SENSORS ---
                'precip_roll_sum_7d': round(precip_7d, 1),
                'precip_roll_sum_30d': round(precip_7d * 3.5, 1), # Approx relation
                'days_since_last_rain': days_no_rain,
                'temp_max_24h': round(np.random.normal(22.3, 2.0), 1), # Around your provided 22.3
                'soil_moisture_root': round(np.random.normal(49.1, 5.0), 1),

                # --- BLOCK 2: SOFT SENSORS ---
                'social_report_count': report_count,
                'social_stress_index': round(report_count / 100.0, 2), # Normalized metric
                'leak_mention_flag': 1 if np.random.random() < 0.3 else 0, # 30% chance of leaks
                'sentiment_polarity': round(np.random.uniform(-0.8, 0.2), 2), # Generally negative regarding water
                'most_common_pain_keyword': np.random.choice(["tandeo", "fuga", "sin agua", "none"]),

                # --- BLOCK 3: SPATIAL/STATIC ---
                # Using your value 9945 as mean, but varying by zone
                'population_density': int(np.random.normal(9945, 2000)),
                'elevation_meters': 2240 if mun == 'Xochimilco' else 2394, # Xochimilco is lower
                'is_weekend': 1 if pd.to_datetime(date).dayofweek >= 5 else 0,
                'month_sin': np.sin(2 * np.pi * pd.to_datetime(date).month / 12),

                # --- BLOCK 4: THE INDICES (YOUR BASELINE) ---
                'msm_drought_level': drought_level, # From your Excel
                'wri_water_risk': round(wri_score, 2), # From WRI logic

                # --- TARGET VARIABLE (What we want to predict) ---
                # 0 = No Shortage, 1 = Shortage Event
                # Logic: High Drought + No Rain + High Reports = Shortage
                'target_shortage_event': 0 # Placeholder, calculated below
            }

            # Ground Truth Logic (Simulating the label for training)
            risk_score = (
                (row['msm_drought_level'] * 0.2) +
                (row['wri_water_risk'] * 0.2) +
                (row['social_stress_index'] * 5.0) -
                (row['precip_roll_sum_7d'] * 0.05)
            )
            row['target_shortage_event'] = 1 if risk_score > 0.8 else 0

            dataset_rows.append(row)

    # Create DataFrame
    df_train = pd.DataFrame(dataset_rows)
    return df_train

# --- EXECUTE ---
if __name__ == "__main__":
    df_cdmx = create_training_dataset()

    print("\n--- DATASET READY FOR XGBOOST ---")
    print(f"Shape: {df_cdmx.shape} (16 Alcaldías x 2 Dates)")
    print("\nSample Rows (First 5):")
    print(df_cdmx[['location_name', 'date', 'social_report_count', 'msm_drought_level', 'target_shortage_event']].head())

    # Save for your model
    df_cdmx.to_csv("cdmx_water_shortage_training_data.csv", index=False)

Generating Training Data for 16 CDMX Zones...

--- DATASET READY FOR XGBOOST ---
Shape: (32, 19) (16 Alcaldías x 2 Dates)

Sample Rows (First 5):
           location_name        date  social_report_count  msm_drought_level  \
0           Azcapotzalco  2025-12-31                    2                  1   
1               Coyoacán  2025-12-31                    7                  2   
2  Cuajimalpa de Morelos  2025-12-31                    4                  2   
3      Gustavo A. Madero  2025-12-31                    7                  0   
4              Iztacalco  2025-12-31                    7                  0   

   target_shortage_event  
0                      0  
1                      1  
2                      1  
3                      0  
4                      1  


In [8]:
import pandas as pd
import numpy as np

# 1. Load Data from the uploaded file
df = pd.read_excel("MunicipiosSequia.xlsx")

# 2. Extract CDMX Municipalities directly from the file
#    (Filters for 'Distrito Federal' or 'Ciudad de México')
cdmx_df = df[df['ENTIDAD'].isin(['Distrito Federal', 'Ciudad de México', 'Ciudad de Mexico'])]
alcaldias_from_file = cdmx_df['NOMBRE_MUN'].unique().tolist()

# 3. Define the specific dates
dates = ['2025-12-31', '2026-01-15']

# 4. Generate the Randomized Risk Vectors
risk_vectors = []
np.random.seed(42)  # Ensures reproducibility

for mun in alcaldias_from_file:
    for date in dates:
        # Randomize Risk Index between 1 (D0) and 5 (D4)
        random_risk = int(np.random.randint(1, 6))

        vector = {
            'location_name': mun,
            'date': date,
            'risk_index': random_risk
        }
        risk_vectors.append(vector)

# 5. Output the result
import pprint
pprint.pprint(risk_vectors)

[{'date': '2025-12-31', 'location_name': 'Azcapotzalco', 'risk_index': 4},
 {'date': '2026-01-15', 'location_name': 'Azcapotzalco', 'risk_index': 5},
 {'date': '2025-12-31', 'location_name': 'Coyoacán', 'risk_index': 3},
 {'date': '2026-01-15', 'location_name': 'Coyoacán', 'risk_index': 5},
 {'date': '2025-12-31',
  'location_name': 'Cuajimalpa de Morelos',
  'risk_index': 5},
 {'date': '2026-01-15',
  'location_name': 'Cuajimalpa de Morelos',
  'risk_index': 2},
 {'date': '2025-12-31', 'location_name': 'Gustavo A. Madero', 'risk_index': 3},
 {'date': '2026-01-15', 'location_name': 'Gustavo A. Madero', 'risk_index': 3},
 {'date': '2025-12-31', 'location_name': 'Iztacalco', 'risk_index': 3},
 {'date': '2026-01-15', 'location_name': 'Iztacalco', 'risk_index': 5},
 {'date': '2025-12-31', 'location_name': 'Iztapalapa', 'risk_index': 4},
 {'date': '2026-01-15', 'location_name': 'Iztapalapa', 'risk_index': 3},
 {'date': '2025-12-31',
  'location_name': 'La Magdalena Contreras',
  'risk_index